In [1]:
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.layers.core import Lambda
from keras.applications.xception import Xception
from keras.layers import Input
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.densenet import DenseNet121
from keras.models import Model
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
import pandas as pd
import os

Using TensorFlow backend.


In [2]:
input_tensor = Input(shape=(224, 224, 3))
tf_inputs = Lambda(lambda x: preprocess_input(x, mode='tf'))(input_tensor)
caffe_inputs = Lambda(lambda x: preprocess_input(x, mode='caffe'))(input_tensor)

base_inception = InceptionV3(input_tensor=input_tensor, weights="imagenet", include_top=True)
inception = Model(inputs=input_tensor, outputs=base_inception(tf_inputs))

base_densenet = DenseNet121(input_tensor=input_tensor, weights="imagenet", include_top=True)
densenet = Model(inputs=input_tensor, outputs=base_densenet(tf_inputs))

base_resnet = ResNet50(input_tensor=input_tensor, weights="imagenet", include_top=True)
resnet = Model(inputs=input_tensor, outputs=base_resnet(caffe_inputs))

base_vgg = VGG16(input_tensor=input_tensor, weights="imagenet", include_top=True)
vgg = Model(inputs=input_tensor, outputs=base_vgg(caffe_inputs))

base_xception = Xception(input_tensor=input_tensor, weights="imagenet", include_top=True)
xception = Model(inputs=input_tensor, outputs=base_xception(tf_inputs))

models = [inception, xception, resnet, densenet, vgg]

for model in models:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
validation_labels = pd.read_csv("imagenet_data/validation_labels.csv", index_col=0)

In [ ]:
validation_labels.head()

In [5]:
experiment_data = pd.read_csv('imagenet_data/exp_data.csv')

In [ ]:
experiment_data.head()

In [ ]:
[model.evaluate(X_exp, Y_exp) for model in models]

In [22]:
all_pictures = set(os.listdir('../adversarial_noise/src/test_data'))
prev_exp = set(experiment_data['name'])
possible_candidates = all_pictures.difference(prev_exp)

In [23]:
len(possible_candidates)

49902

In [11]:
X_supp = []
Y_supp = []
img_names = []
img_labels = []
count = 0
num_images = 100
num_models = len(models)
prev_count = 0
    
for img_name in prev_exp:
    label = validation_labels.loc[validation_labels['name'] == img_name, 'label'].values[0]
    img_path = "imagenet_pictures/" + img_name
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    y = (np.arange(1000) == label).astype(np.float32)

    preds = [np.argmax(model.predict(x)[0]) == label for model in models]
    num_correct = sum(preds)

    if sum(preds) == num_models:
        X_supp.append(x[0])
        Y_supp.append(y)
        img_names.append(img_name)
        img_labels.append(label)
        count += 1
        prev_count += 1
        print count, prev_count

1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96


In [13]:
len(possible_candidates)

0

In [25]:
X_supp2 = []
Y_supp2 = []

for img_name in possible_candidates:
    label = validation_labels.loc[validation_labels['name'] == img_name, 'label'].values[0]
    img_path = "../adversarial_noise/src/test_data/" + img_name
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    y = (np.arange(1000) == label).astype(np.float32)

    preds = [np.argmax(model.predict(x)[0]) == label for model in models]
    num_correct = sum(preds)

    if sum(preds) == num_models:
        X_supp2.append(x[0])
        Y_supp2.append(y)
        img_names.append(img_name)
        img_labels.append(label)
        count += 1
        print count

    if count >= num_images:
        break
        
X_supp2 = np.array(X_supp2)
Y_supp2 = np.array(Y_supp2)    

97
98
99
100


In [30]:
X_exp = np.concatenate([X_supp, X_supp2])
Y_exp = np.concatenate([Y_supp, Y_supp2])

In [32]:
rows = []
for img_name in img_names:
    rows.append(experiment_data[experiment_data.name == img_name])

In [66]:
t = list(experiment_data[experiment_data.name.isin(img_names)].values.tolist())

In [67]:
others = set(img_names).difference(set(experiment_data[experiment_data.name.isin(img_names)].name))

In [68]:
experiment_data.head()

,name,true_label,true_label_description,target_label,target_label_description
0,ILSVRC2012_val_00012589.JPEG,505,coffeepot,230,"Shetland sheepdog, Shetland sheep dog, Shetland"
1,ILSVRC2012_val_00009302.JPEG,595,"harvester, reaper",83,"prairie chicken, prairie grouse, prairie fowl"
2,ILSVRC2012_val_00032256.JPEG,915,yurt,917,comic book
3,ILSVRC2012_val_00030794.JPEG,989,"hip, rose hip, rosehip",832,"stupa, tope"
4,ILSVRC2012_val_00005030.JPEG,146,"albatross, mollymawk",707,"pay-phone, pay-station"


In [64]:
imagenet_labels = pd.read_csv('imagenet_labels.csv')

In [65]:
imagenet_labels.

,label,description
0,0,"tench, Tinca tinca"
1,1,"goldfish, Carassius auratus"
2,2,"great white shark, white shark, man-eater, man..."
3,3,"tiger shark, Galeocerdo cuvieri"
4,4,"hammerhead, hammerhead shark"


In [69]:
others = img_names[-4:]

In [70]:
for name in others:
    label = validation_labels[validation_labels.name == name].label.values[0]
    description = imagenet_labels[imagenet_labels.label == label].description.values[0]
    t.append([name, label, description, None, None])

In [75]:
experiment_data = pd.DataFrame(t, columns=experiment_data.columns)

In [76]:
[model.evaluate(X_exp, Y_exp) for model in models]

100/100 [==============================] - 132s 1s/step


[[0.024925113897770645, 1.0],
 [0.03127027690410614, 1.0],
 [0.0739623986184597, 1.0],
 [0.059462333172559736, 1.0],
 [0.09292019605636596, 1.0]]

In [78]:
experiment_data.to_csv('imagenet_data/imagenet_experiment_data_description.csv', index=False)

In [80]:
np.save('imagenet_data/X_exp.npy', X_exp)
np.save('imagenet_data/Y_exp.npy', Y_exp)

In [2]:
import numpy as np

In [4]:
X_exp = np.load('dl_experiments_data/imagenet/X_exp.npy')
Y_exp = np.load('dl_experiments_data/imagenet/Y_exp.npy')

In [5]:
[model.evaluate(X_exp, Y_exp) for model in models]

100/100 [==============================] - 90s 905ms/step


[[0.024925113897770645, 1.0],
 [0.03127027690410614, 1.0],
 [0.0739623986184597, 1.0],
 [0.059462333172559736, 1.0],
 [0.09292019605636596, 1.0]]

In [6]:
X_exp[4,5,5,1]

96.0